# Laphet (Version 0.7) with myPOS Tags

Language model ကို NLP task အမျိုးမျိုးအတွက် အသုံးပြုကြပါတယ်။ ဒီ notebook မှာတော့ Laphet LM Toolkit ကို သုံးပြီး မြန်မာစာကြောင်းတွေရဲ့ Part-of-Speech prediction/generation ကို လက်တွေ့ လုပ်ပြပါမယ်။  

Version 0.7 မှာက embedding ကို fasttext နဲ့ပါ သုံးအောင် update လုပ်ထားတာပါ။ အဲဒါကြောင့် --embedding_method နဲ့ --fasttext_model ဆိုတဲ့ option တွေက အသစ်တွေပါ။ 

In [68]:
!python ./laphet.py --help

usage: laphet.py [-h] --model_type {mlp,bilstm,transformer,bert,gpt} [--train]
                 [--generate] [--test] [--data DATA] [--model MODEL]
                 [--vocab VOCAB] [--dev_file DEV_FILE] [--test_file TEST_FILE]
                 [--prompt PROMPT] [--input INPUT] [--seq_len SEQ_LEN]
                 [--output OUTPUT] [--no_of_generation NO_OF_GENERATION]
                 [--epochs EPOCHS] [--batch_size BATCH_SIZE] [--lr LR]
                 [--embed_dim EMBED_DIM] [--num_heads NUM_HEADS]
                 [--num_layers NUM_LAYERS] [--hidden_dim HIDDEN_DIM]
                 [--ff_dim FF_DIM] [--dropout DROPOUT]
                 [--temperature TEMPERATURE] [--top_k TOP_K] [--top_p TOP_P]
                 [--embedding_method {nn.Embedding,fasttext_freeze,fasttext_no_freeze}]
                 [--fasttext_model FASTTEXT_MODEL]

Laphet language model toolkit for Burmese.

options:
  -h, --help            show this help message and exit
  --model_type {mlp,bilstm,transformer,bert

## Dataset Information

Link: [https://github.com/ye-kyaw-thu/myPOS](https://github.com/ye-kyaw-thu/myPOS)  

Preprocessing အနေနဲ့ myPOS dataset ထဲက tag တွေကိုပဲ ဆွဲထုတ်ယူထားပါတယ်။  
tag တွေ သို့မဟုတ် word တွေကို ဆွဲထုတ်ဖို့အတွက် သုံးခဲ့တဲ့ perl code က ဒီလင့်ကနေ ရယူနိုင်ပါတယ်။  

[https://github.com/ye-kyaw-thu/myPOS/blob/master/corpus-draft-ver-1.0/mk-wordtag.pl](https://github.com/ye-kyaw-thu/myPOS/blob/master/corpus-draft-ver-1.0/mk-wordtag.pl)  

Laphet repository မှာလည်း ကော်ပီကူးထည့်ပေးထားပါမယ်။  

In [57]:
!ls -lh ./data/myPOS/tag/*

-rw-rw-r-- 1 ye ye 103K Jan 26 18:03 ./data/myPOS/tag/dev_tag.txt
-rw-rw-r-- 1 ye ye   95 Jan 24 18:42 ./data/myPOS/tag/start_tags.txt
-rw-rw-r-- 1 ye ye  48K Jan 24 17:43 ./data/myPOS/tag/test_tag.txt
-rw-rw-r-- 1 ye ye 1.9M Jan 26 18:02 ./data/myPOS/tag/train_tag.txt


In [58]:
!wc ./data/myPOS/tag/train_tag.txt

  40000  522381 1908736 ./data/myPOS/tag/train_tag.txt


In [59]:
!wc ./data/myPOS/tag/dev_tag.txt

  2196  28668 104680 ./data/myPOS/tag/dev_tag.txt


In [60]:
!wc ./data/myPOS/tag/test_tag.txt

 1000 13468 48966 ./data/myPOS/tag/test_tag.txt


In [61]:
!wc ./data/myPOS/tag/start_tags.txt

15 28 95 ./data/myPOS/tag/start_tags.txt


## Data Format

In [62]:
!head ./data/myPOS/tag/train_tag.txt

num n v n ppm n num part v ppm punc
n part ppm v v part v part part n v n conj punc n conj n n ppm v part part n n part v part n conj n n v ppm punc
adj n ppm v part n part v ppm punc
n v part ppm v v part part n n part ppm v n part ppm n ppm adv v part v part part part punc
pron ppm pron n part punc
num n n n n punc num part punc n n punc n n punc
pron n tn part v part part ppm punc
pron part ppm pron n ppm n ppm v part v part ppm punc
n v part adj n ppm pron part part punc
adj n ppm n punc n punc n part n part v ppm punc


In [63]:
!tail ./data/myPOS/tag/dev_tag.txt

pron tn part part v ppm punc
n n n n part ppm adv v part part ppm punc
conj n part v conj adv v part part punc
pron pron v part n v part n v part part punc
n v part ppm n v ppm punc
v part ppm punc
v v part v part punc
pron ppm n n ppm v part part part part punc
pron pron part ppm n tn part part ppm n v part v part punc
pron part ppm adj n part punc


In [64]:
!head ./data/myPOS/tag/test_tag.txt

tn n ppm n tn part punc
n ppm pron pron ppm v part ppm punc
pron n v v part punc
n v part v ppm part punc
n adv v part punc tn tn n part v part conj v part ppm part punc
v part conj n ppm v part v ppm punc
n ppm v ppm punc
n v conj n v part part part punc
pron v part ppm tn part part v part ppm punc
n part ppm v conj n part ppm n n ppm n v ppm punc


In [65]:
!cat ./data/myPOS/tag/start_tags.txt

pron
n
adj
v
pron part
pron ppm
n v
n n
v part
n part
pron pron
pron ppm
n tn
adj v part
n n n


start_tags.txt ဖိုင်မှာ start word (တကယ်ကတော့ ဒီနေရာမှာ start tag ပါ) အနေနဲ့ ပေးတဲ့အခါမှာ တစ်လုံးထက်မက ပိုပေးထားတာကို ဂရုပြုပါ။  

## Using --nn.Embedding

ဒီ notebook မှာက အရင် version 0.6 မှာလိုပဲ simple embedding ဖြစ်တဲ့ nn.Embedding နဲ့ပဲ သွားပါမယ်။   

## Bash Shell Script

Experiment အကုန်၊ ဆိုလိုတာက MLP, Bi-LSTM, Transformer, BERT, GPT မော်ဒယ်အကုန်နဲ့ experiment လုပ်မယ် ဆိုရင်တော့ အောက်ပါ shel script နဲ့ run ပါ။  

In [66]:
!cat ./train_test_tag.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 28 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 \
      --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u laphet

ဒီ notebook မှာ အကုန်လုံးကို တန်းစီပြီး တခါတည်း run ချလိုက်ရင် လိုက်ကြည့်ရခက်နေမှာစိုးလို့ မော်ဒယ် တစ်ခုချင်းစီကို သပ်သပ်စီ run ပြသွားပါမယ်။ အဲဒီလို လုပ်ဖို့အတွက် ဆိုရင် task line တွေကို comment အပိတ်အဖွင့်လုပ်ပြီး ကစားပေးရပါလိမ့်မယ်။  

## Bash Script for MLP  

In [69]:
!cat ./train_test_tag.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 28 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 \
      --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u laphet

## MLP based LM Training, Text Generation and Testing

In [71]:
!./train_test_tag.sh

Training Mlp language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:09<00:00, 136.94it/s]
Epoch 1, Training Loss: 0.6107
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 507.72it/s]
Epoch 1, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:08<00:00, 139.86it/s]
Epoch 2, Training Loss: 0.5729
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 521.82it/s]
Epoch 2, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:09<00:00, 138.85it/s]
Epoch 3, Training Loss: 0.5729
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 482.32it/s]
Epoch 3, Validation Loss: 0.5729
Best model saved at ./model/tag/mlp.model with validation loss: 0.5729
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:08<00:00, 140.91it/s]

## GPU Usage for MLP-based Modeling

## Checking Model, Output and Log

In [72]:
!tree ./model/tag/

./model/tag/
├── mlp.model
└── mlp.model.vocab

1 directory, 2 files


In [73]:
!ls -lh ./model/tag/*

-rw-rw-r-- 1 ye ye 568K Jan 28 14:57 ./model/tag/mlp.model
-rw-rw-r-- 1 ye ye  110 Jan 28 14:56 ./model/tag/mlp.model.vocab


In [74]:
!wc ./model/tag/mlp.model.vocab

 17  34 110 ./model/tag/mlp.model.vocab


In [75]:
!cat ./model/tag/mlp.model.vocab

num	0
abb	1
ppm	2
v	3
part	4
[UNK]	5
sb	6
[PAD]	7
conj	8
pron	9
fw	10
int	11
adv	12
adj	13
n	14
tn	15
punc	16


## Tag Generation Results

In [76]:
!cat ./data/myPOS/tag/start_tags.txt

pron
n
adj
v
pron part
pron ppm
n v
n n
v part
n part
pron pron
pron ppm
n tn
adj v part
n n n


အထက်ပါ input ဖိုင်ထဲက start tag တွေကို အခြေခံပြီး MLP-based LM က generated လုပ်ပြီးထွက်လာတဲ့ output က အောက်ပါအတိုင်းပါ။  

In [77]:
!cat -n ./output/tag/mlp_gen_texts.txt

     1	pron sb [UNK]
     2	pron fw [UNK]
     3	pron v fw
     4	pron v [PAD]
     5	pron ppm int
     6	n adv punc
     7	n pron pron
     8	n conj sb
     9	n ppm num
    10	n adj pron
    11	adj ppm [UNK]
    12	adj ppm [PAD]
    13	adj part [PAD]
    14	adj v ppm
    15	adj n v
    16	v [PAD] num
    17	v int sb
    18	v abb fw
    19	v tn n
    20	v ppm fw
    21	pron part v ppm
    22	pron part [PAD] [PAD]
    23	pron part int [PAD]
    24	pron part adj v
    25	pron part sb [PAD]
    26	pron ppm part tn
    27	pron ppm n sb
    28	pron ppm ppm adv
    29	pron ppm punc int
    30	pron ppm int n
    31	n v [PAD] pron
    32	n v adj adv
    33	n v [PAD] fw
    34	n v abb sb
    35	n v adj part
    36	n n [UNK] punc
    37	n n conj int
    38	n n ppm n
    39	n n num n
    40	n n tn ppm
    41	v part pron sb
    42	v part sb fw
    43	v part [PAD] n
    44	v part n num
    45	v part conj adj
    46	n part int conj
    47	n part part abb
    48	n part tn num
    49	n part [UNK] abb


tag generation ကို ဥပမာ အလုံး ၃၀ ထားကြည့်ပြီး run ကြည့်ရအောင်။  

In [78]:
!time python laphet.py --model_type mlp --generate --model ./model/tag/mlp.model \
--seq_len 30 --input ./data/myPOS/tag/start_tags.txt --no_of_generation 1 --embedding_method nn.Embedding

/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: pron conj num part [UNK] num pron

Test data နဲ့ evaluation လုပ်ထားတဲ့ ရလဒ်ကို ကြည့်ကြည့်ရအောင်။  

In [79]:
!tail -n 2 ./log/tag/mlp.log

Average Perplexity on Test Data: 1.1039
Average Cross-Entropy on Test Data: 0.0988


## Bi-LSTM based LM Building, Tag Generation and Testing

bash shell script ကို အောက်ပါအတိုင်း updated လုပ်ခဲ့။  

# Run tasks for each model type in the specified order
\#task mlp  
task bilstm  
\#task transformer  
\#task bert  
\#task gpt  

In [80]:
!./train_test_tag.sh

Training Bilstm language model:
Epoch 1/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 58.25it/s]
Epoch 1, Training Loss: 0.1714
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 177.90it/s]
Epoch 1, Validation Loss: 0.0013
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0013
Epoch 2/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 59.17it/s]
Epoch 2, Training Loss: 0.0011
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 166.91it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|████████████████| 1250/1250 [00:20<00:00, 59.62it/s]
Epoch 3, Training Loss: 0.0003
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 168.17it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0000
Epoch 4/10 (Training): 100%|████████████████| 1250/1250 [00:21<00:00

## Training/Testing with 30 epochs

In [81]:
!cat train_test_tag.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 28 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 30 --batch_size 32 \
      --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u laphet

In [82]:
!./train_test_tag.sh

Training Bilstm language model:
Epoch 1/30 (Training): 100%|████████████████| 1250/1250 [00:21<00:00, 58.75it/s]
Epoch 1, Training Loss: 0.1783
Epoch 1/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 169.08it/s]
Epoch 1, Validation Loss: 0.0019
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0019
Epoch 2/30 (Training): 100%|████████████████| 1250/1250 [00:20<00:00, 59.76it/s]
Epoch 2, Training Loss: 0.0013
Epoch 2/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 174.59it/s]
Epoch 2, Validation Loss: 0.0002
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0002
Epoch 3/30 (Training): 100%|████████████████| 1250/1250 [00:20<00:00, 60.76it/s]
Epoch 3, Training Loss: 0.0003
Epoch 3/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 180.85it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/bilstm.model with validation loss: 0.0000
Epoch 4/30 (Training): 100%|████████████████| 1250/1250 [00:20<00:00

## Updated the Shell Script

Text generation, testing ပဲ လုပ်ဖို့နဲ့ sequence length ကို 5 ပဲ ထားပြီး run ကြည့်မယ်။  

In [84]:
!cat ./train_test_tag.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 28 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    #echo "Training ${model_type^} language model:";
    #time python -u laphet.py --model_type $model_type --train --data $train_data \
    #  --dev_file $dev_data --model $model_file --seq_len 50 --epochs 30 --batch_size 32 \
    #  --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u la

## Training Again for Bi-LSTM

In [85]:
!./train_test_tag.sh

Text generation:
/home/ye/exp/name-lm/lib/laphet.py:228: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
All logits are -Inf after filterin

## Updated the Bash Shell Script

In [86]:
!cat ./train_test_tag.sh

#!/bin/bash

# Updated for Laphet LM Toolkit Version 0.7
# Last updated: 28 Jan 2025

# Create the output and log directories if they don't exist
mkdir -p model/tag/
mkdir -p output/tag/
mkdir -p log/tag/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/tag/${model_type}.model"
  local output_file="./output/tag/${model_type}_gen_texts.txt"
  local log_file="./log/tag/${model_type}.log"
  local train_data="./data/myPOS/tag/train_tag.txt"
  local dev_data="./data/myPOS/tag/dev_tag.txt"
  local test_data="./data/myPOS/tag/test_tag.txt"
  local start_tag="./data/myPOS/tag/start_tags.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 30 --batch_size 32 \
      --lr 0.0001 --embedding_method nn.Embedding;

    echo "Text generation:";
    time python -u laphet

## Transformer based LM Training, Tag Generation and Testing

In [87]:
!./train_test_tag.sh

Training Transformer language model:
Epoch 1/30 (Training): 100%|███████████████| 1250/1250 [00:06<00:00, 207.39it/s]
Epoch 1, Training Loss: 0.0261
Epoch 1/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 591.43it/s]
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/transformer.model with validation loss: 0.0004
Epoch 2/30 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 221.10it/s]
Epoch 2, Training Loss: 0.0003
Epoch 2/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 627.57it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/transformer.model with validation loss: 0.0001
Epoch 3/30 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 219.67it/s]
Epoch 3, Training Loss: 0.0001
Epoch 3/30 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 592.04it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/transformer.model with validation loss: 0.0000
Epoch 4/30 (Training): 100%|███████████████| 125

## GPU Usage of Transformer

## Checking Model, Generated Tags and Log

In [88]:
!tree ./model/tag/

./model/tag/
├── bilstm.model
├── bilstm.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 6 files


In [89]:
!ls -lh ./model/tag/transformer.model*

-rw-rw-r-- 1 ye ye 8.2M Jan 28 17:22 ./model/tag/transformer.model
-rw-rw-r-- 1 ye ye  110 Jan 28 17:20 ./model/tag/transformer.model.vocab


In [90]:
!tree ./output/tag/

./output/tag/
├── bilstm_gen_texts.txt
├── mlp_gen_texts.txt
└── transformer_gen_texts.txt

1 directory, 3 files


In [91]:
!cat -n ./output/tag/transformer_gen_texts.txt

     1	pron pron tn
     2	pron [UNK] ppm
     3	pron int v
     4	pron ppm conj
     5	pron part conj
     6	n int ppm
     7	n [PAD] n
     8	n ppm v
     9	n fw ppm
    10	n adv adj
    11	adj ppm int
    12	adj sb int
    13	adj adj [PAD]
    14	adj fw num
    15	adj v adv
    16	v abb num
    17	v v tn
    18	v v v
    19	v sb sb
    20	v part [UNK]
    21	pron part [UNK] adj
    22	pron part [UNK] adj
    23	pron part punc fw
    24	pron part pron pron
    25	pron part ppm pron
    26	pron ppm adv [UNK]
    27	pron ppm [UNK] adj
    28	pron ppm n conj
    29	pron ppm v fw
    30	pron ppm sb adj
    31	n v tn adv
    32	n v adv num
    33	n v adv punc
    34	n v adv fw
    35	n v sb n
    36	n n adv tn
    37	n n [PAD] pron
    38	n n fw part
    39	n n pron fw
    40	n n punc [PAD]
    41	v part tn fw
    42	v part [PAD] pron
    43	v part ppm num
    44	v part sb int
    45	v part [PAD] [PAD]
    46	n part [UNK] adj
    47	n part pron adj
    48	n part v [PAD]
    49	n part conj

In [92]:
!tail -n 2 ./log/tag/transformer.log

Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000


## Updated Bash Shell Script

\#task mlp  
\#task bilstm  
\#task transformer  
task bert  
\#task gpt  

## BERT based LM Building, Tag Generation and Testing

In [93]:
!./train_test_tag.sh

Training Bert language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 214.68it/s]
Epoch 1, Training Loss: 0.0303
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 629.24it/s]
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/bert.model with validation loss: 0.0004
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 222.20it/s]
Epoch 2, Training Loss: 0.0004
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 628.19it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 223.03it/s]
Epoch 3, Training Loss: 0.0001
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 628.81it/s]
Epoch 3, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 224.33i

## GPU Usage of BERT

## Checking Model, Generated Tags and Logs

In [94]:
!tree ./model/tag/

./model/tag/
├── bert.model
├── bert.model.vocab
├── bilstm.model
├── bilstm.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 8 files


In [95]:
!ls -lh ./model/tag/*.model

-rw-rw-r-- 1 ye ye 8.2M Jan 28 17:37 ./model/tag/bert.model
-rw-rw-r-- 1 ye ye  85M Jan 28 15:32 ./model/tag/bilstm.model
-rw-rw-r-- 1 ye ye 568K Jan 28 14:57 ./model/tag/mlp.model
-rw-rw-r-- 1 ye ye 8.2M Jan 28 17:22 ./model/tag/transformer.model


In [96]:
!cat -n ./output/tag/bert_gen_texts.txt

     1	pron abb tn
     2	pron abb part
     3	pron adv tn
     4	pron adj ppm
     5	pron n part
     6	n conj fw
     7	n part adv
     8	n abb v
     9	n pron pron
    10	n ppm tn
    11	adj [UNK] ppm
    12	adj n [UNK]
    13	adj [PAD] [PAD]
    14	adj adv adv
    15	adj int adv
    16	v conj punc
    17	v fw adv
    18	v punc [UNK]
    19	v sb abb
    20	v [PAD] conj
    21	pron part [PAD] adv
    22	pron part punc adv
    23	pron part ppm sb
    24	pron part fw sb
    25	pron part sb abb
    26	pron ppm conj ppm
    27	pron ppm adj punc
    28	pron ppm adj conj
    29	pron ppm part int
    30	pron ppm fw adj
    31	n v pron conj
    32	n v num punc
    33	n v abb conj
    34	n v [PAD] sb
    35	n v adj pron
    36	n n adj ppm
    37	n n conj pron
    38	n n part int
    39	n n [PAD] punc
    40	n n conj ppm
    41	v part ppm num
    42	v part adj part
    43	v part fw punc
    44	v part tn pron
    45	v part conj adj
    46	n part punc sb
    47	n part part n
    48	n part tn int

In [97]:
!cat ./log/tag/bert.log

Training Bert language model:
Epoch 1, Training Loss: 0.0303
Epoch 1, Validation Loss: 0.0004
Best model saved at ./model/tag/bert.model with validation loss: 0.0004
Epoch 2, Training Loss: 0.0004
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 3, Training Loss: 0.0001
Epoch 3, Validation Loss: 0.0001
Best model saved at ./model/tag/bert.model with validation loss: 0.0001
Epoch 4, Training Loss: 0.0001
Epoch 4, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 5, Training Loss: 0.0000
Epoch 5, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 6, Training Loss: 0.0000
Epoch 6, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 7, Training Loss: 0.0000
Epoch 7, Validation Loss: 0.0000
Best model saved at ./model/tag/bert.model with validation loss: 0.0000
Epoch 8, Training 

## Updated Bash Shell Script

\#task mlp  
\#task bilstm  
\#task transformer  
\#task bert  
task gpt  

## GPT based LM Training, Tag Generation and Testing

In [98]:
!./train_test_tag.sh

Training Gpt language model:
Epoch 1/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 220.21it/s]
Epoch 1, Training Loss: 0.0130
Epoch 1/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 622.94it/s]
Epoch 1, Validation Loss: 0.0002
Best model saved at ./model/tag/gpt.model with validation loss: 0.0002
Epoch 2/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 235.25it/s]
Epoch 2, Training Loss: 0.0002
Epoch 2/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 604.23it/s]
Epoch 2, Validation Loss: 0.0001
Best model saved at ./model/tag/gpt.model with validation loss: 0.0001
Epoch 3/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 230.58it/s]
Epoch 3, Training Loss: 0.0001
Epoch 3/10 (Validation): 100%|█████████████████| 69/69 [00:00<00:00, 592.55it/s]
Epoch 3, Validation Loss: 0.0000
Best model saved at ./model/tag/gpt.model with validation loss: 0.0000
Epoch 4/10 (Training): 100%|███████████████| 1250/1250 [00:05<00:00, 235.87it/s]

## GPU Usage of GPT LM Modeling

## Checking Model, Generated Tags, Log

In [99]:
!tree ./model/tag/

./model/tag/
├── bert.model
├── bert.model.vocab
├── bilstm.model
├── bilstm.model.vocab
├── gpt.model
├── gpt.model.vocab
├── mlp.model
├── mlp.model.vocab
├── transformer.model
└── transformer.model.vocab

1 directory, 10 files


In [100]:
!ls -lh ./model/tag/gpt*

-rw-rw-r-- 1 ye ye 8.2M Jan 28 17:40 ./model/tag/gpt.model
-rw-rw-r-- 1 ye ye  110 Jan 28 17:40 ./model/tag/gpt.model.vocab


In [101]:
!cat -n ./output/tag/gpt_gen_texts.txt

     1	pron punc tn
     2	pron n abb
     3	pron v adj
     4	pron tn v
     5	pron adv [PAD]
     6	n fw abb
     7	n sb int
     8	n [UNK] punc
     9	n tn abb
    10	n [PAD] punc
    11	adj [PAD] conj
    12	adj v adj
    13	adj n ppm
    14	adj int sb
    15	adj num tn
    16	v v n
    17	v adv n
    18	v punc part
    19	v int int
    20	v int abb
    21	pron part pron int
    22	pron part [UNK] v
    23	pron part pron fw
    24	pron part n n
    25	pron part n ppm
    26	pron ppm v [UNK]
    27	pron ppm adj adj
    28	pron ppm int abb
    29	pron ppm abb adj
    30	pron ppm abb abb
    31	n v num abb
    32	n v punc [UNK]
    33	n v abb abb
    34	n v pron [UNK]
    35	n v pron part
    36	n n [UNK] n
    37	n n conj part
    38	n n [PAD] fw
    39	n n adv adv
    40	n n abb abb
    41	v part sb conj
    42	v part ppm abb
    43	v part part ppm
    44	v part sb fw
    45	v part n part
    46	n part num part
    47	n part pron n
    48	n part n conj
    49	n part int v
    50	n p

In [102]:
!tail -n 2 ./log/tag/*.log

==> ./log/tag/bert.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/bilstm.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/gpt.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000

==> ./log/tag/mlp.log <==
Average Perplexity on Test Data: 1.1039
Average Cross-Entropy on Test Data: 0.0988

==> ./log/tag/transformer.log <==
Average Perplexity on Test Data: 1.0000
Average Cross-Entropy on Test Data: 0.0000
